**Корректность проверена на Python 3.6:**
+ pandas 0.23.4
+ numpy 1.15.4
+ sklearn 0.20.2

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [2]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [3]:
iris = datasets.load_iris()

In [4]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [5]:
classifier = linear_model.SGDClassifier(random_state = 0, tol=1e-3)

### Генерация сетки

In [6]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [7]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [8]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [9]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [10]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 11.8 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             estimator=SGDClassifier(random_state=0),
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                                  'squared_loss'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [11]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, max_iter=9, penalty='l1',
              random_state=0)

In [12]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


In [15]:
grid_cv.cv_results_

{'mean_fit_time': array([0.0169111 , 0.00312493, 0.00625031, 0.00468795, 0.00312498,
        0.00449746, 0.00410261, 0.00430264, 0.0042028 , 0.00430274,
        0.00430286, 0.00470295, 0.00500329, 0.00410242, 0.00490317,
        0.00480316, 0.00490303, 0.00490332, 0.00486817, 0.00312524,
        0.00468776, 0.00156255, 0.00468762, 0.0031251 , 0.00468791,
        0.00312498, 0.00312519, 0.00312521, 0.003125  , 0.00468791,
        0.00312486, 0.00468776, 0.00468779, 0.00300143, 0.00352633,
        0.00420604, 0.003125  , 0.00581567, 0.00500329, 0.00500345,
        0.00500319, 0.00360248, 0.00490324, 0.00350239, 0.00440295,
        0.00470302, 0.00480318, 0.00450296, 0.00490317, 0.00450313,
        0.00490315, 0.00450275, 0.00490346, 0.00480313, 0.00470309,
        0.00490324, 0.0049032 , 0.00390258, 0.00460281, 0.00330224,
        0.00440283, 0.0048032 , 0.00480316, 0.00450311, 0.00470297,
        0.00400236, 0.00460296, 0.00450284, 0.00380208, 0.00350204,
        0.0033021 , 0.00350223,

#### Randomized grid search

In [37]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [38]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

CPU times: user 353 ms, sys: 0 ns, total: 353 ms
Wall time: 351 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
          error_score='raise-deprecating',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=0, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=None,
          param_distributions={'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'penalty': ['l1', 'l2'], 'max_iter': array([5, 6, 7, 8, 9]), 'alpha': array([0.0001 , 0.00032, 0.00055, 0.00078, 0.001  ])},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [39]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666667
{'penalty': 'l1', 'max_iter': 9, 'loss': 'log', 'alpha': 0.00055}


In [40]:
#!/usr/bin/python

from datetime import datetime


def control_sum(str):
    return sum(map(ord, list(str)))


def check(reply):
    def _is_number(str):
        try:
            int(str)
            return True
        except ValueError:
            return False

    if "Control sum:" not in reply:
        return False
    parts = reply.split("Control sum:")
    received_current_time = parts[0].strip()
    received_control_sum = parts[1].strip()
    if not _is_number(received_control_sum):
        return False
    else:
        received_control_sum = int(received_control_sum)
    expected_control_sum = control_sum(received_current_time)
    return expected_control_sum == received_control_sum


current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)
print("Control sum: " + str(control_sum(current_time)))

#print check(current_time + '\n' + "Control sum: " + str(control_sum(current_time)))

2019-01-16 11:14:39
Control sum: 949
